In [15]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import os, sys, inspect

import pywikibot as pb
from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
from mako.template import Template
from urllib.request import Request, urlopen
from urllib.parse import quote

import re

commons_site = pb.Site("commons", "commons")

In [16]:
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy

req_proxy = RequestProxy()

2018-01-10 10:56:42,767 root   DEBUG    === Initialized Proxy Parsers ===
2018-01-10 10:56:42,768 root   DEBUG    	 FreeProxy Parser of 'http://free-proxy-list.net' with required bandwidth: '150' KBs
2018-01-10 10:56:42,769 root   DEBUG    	 ProxyForEU Parser of 'http://proxyfor.eu/geo.php' with required bandwidth: '1.0' KBs
2018-01-10 10:56:42,776 root   DEBUG    	 RebroWeebly Parser of 'http://rebro.weebly.com' with required bandwidth: '150' KBs
2018-01-10 10:56:42,777 root   DEBUG    	 SemairProxy Parser of 'https://premproxy.com/list/' with required bandwidth: '150' KBs
2018-01-10 10:56:42,778 root   DEBUG    =================================
2018-01-10 10:56:43,260 http_request_randomizer.requests.parsers.FreeProxyParser DEBUG    Proxy Invalid: <zip object at 0x7f37b21d81c8>
2018-01-10 10:56:43,601 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.178
2018-01-10 10:56:43,602 http_request_randomizer.requests.parsers.ProxyForEuParser DEBU

In [29]:
url = 'http://www.lamoncloa.gob.es/multimedia/fototeca/Paginas/Imagen.aspx?img=08039dc131'
categories = ['María Teresa Fernández de la Vega', 
              'José Enrique Serrano Martínez',
              'April 2007 in Madrid',
              'Palacio de la Moncloa']
cat_string = '\n'.join(['[[Category:'+i+']]' for i in categories])

In [30]:
while True:
    request = req_proxy.generate_proxied_request(url)
    if request is not None:
        #print("{0}".format(u''.join(request.text).encode('utf-8')))
        response = request.text
        break
    else :
        print ('Trying')
soup = BeautifulSoup(response, 'html.parser')

2018-01-10 13:07:06,024 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (X11; U; Linux x86_64; it; rv:1.9.1.9) Gecko/20100402 Ubuntu/9.10 (karmic) Firefox/3.5.9 (.NET CLR 3.5.30729'}
2018-01-10 13:07:06,028 root   DEBUG    Using proxy: 178.206.218.7:8080 | FreeProxy
2018-01-10 13:07:36,041 root   DEBUG    Proxy unreachable - Removed Straggling proxy: 178.206.218.7:8080 | FreeProxy PL Size = 1301
2018-01-10 13:07:36,046 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Opera/9.80 (Windows NT 5.1; U; it) Presto/2.7.62 Version/11.0'}
2018-01-10 13:07:36,047 root   DEBUG    Using proxy: 174.97.107.240:8118 | FreeProxy


Trying


2018-01-10 13:08:06,080 root   DEBUG    Proxy unreachable - Removed Straggling proxy: 174.97.107.240:8118 | FreeProxy PL Size = 1300
2018-01-10 13:08:06,085 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Ubuntu/10.10 Chromium/10.0.648.0 Chrome/10.0.648.0 Safari/534.1'}
2018-01-10 13:08:06,091 root   DEBUG    Using proxy: 5.69.170.246:8080 | FreeProxy
2018-01-10 13:08:06,190 root   DEBUG    Proxy unreachable - Removed Straggling proxy: 5.69.170.246:8080 | FreeProxy PL Size = 1299
2018-01-10 13:08:06,195 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; ja-JP) AppleWebKit/533.16 (KHTML, like Gecko) Version/5.0 Safari/533.1'}
2018-01-10 13:08:06,196 root   DEBUG    Using proxy: 38.64.129.242:8080 | FreeProxy


Trying
Trying
RR Status 200


In [31]:
title = soup.find_all("h1", class_="contenidoTitulo")[0].get_text().strip().replace(':', ' - ').replace('  ', ' ')
title

'Fernández de la Vega preside la reunión sobre la violencia de genero con los subdelegados de Gobierno'

In [32]:
import dateparser as dp
import datetime

try:
    pub_date_sp = ' '.join(soup.find_all("div", class_="contenidoFecha")[0].get_text().strip().split(' ')[-4:])
    parsed_pub_date = dp.parse(pub_date_sp)
    pub_date = parsed_pub_date.strftime('%Y-%m-%d')
except :
    pub_date = None

pub_date

'2007-04-10'

In [33]:
contents = soup.find_all("div", class_="entradillaContenedor")[0].get_text().strip().replace('\t', '').split('\r\n')
contents = [item.replace('\n', '') for item in contents if len(item)>0]
author = contents[0]
author

'Pool Moncloa'

In [34]:
description = contents[1]
description

'María Teresa Fernández de la Vega, vicepresidenta primera, ministra de la Presidencia y portavoz del Gobierno, preside la reunión sobre la violencia de genero con los subdelegados de Gobierno junto a José Enrique Serrano Martínez, director del Gabinete del Presidente.'

In [35]:
image_url = 'http://www.lamoncloa.gob.es' + soup.find_all("a", id="ctl00_PlaceHolderMain_DisplayMode_aDescargar")[0]['href']
image_url

'http://www.lamoncloa.gob.es/multimedia/fototeca/Paginas/ImageF.axd?img=70d716c531'

In [36]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Fototeca: ${title}]
|author=Ministerio de la Presidencia. Gobierno de España (${author})
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
${cat_string}"""

In [37]:
from PIL import Image
from io import BytesIO
import requests

print ("Image to download: {0}".format(image_url))
image_name = "%s.jpg" %(title)
image_path = os.path.join(os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0])), image_name)
print ("Image downloaded to: {0}".format(image_path))
while True:
    print ('Trying to download')
    img_request = req_proxy.generate_proxied_request(image_url)
    if img_request is not None:
        bytes = BytesIO(img_request.content)
        image = Image.open(bytes)
        image.save(image_path)
        print ('Image downloaded and accessed')
        break
    else :
        print ('Trying again...')

vars = {
    "url": url,
    "description": description,
    "author": author,
    "date": pub_date,
    "title": title,
    "cat_string": cat_string
}
t = Template(template)
_text = t.render(**vars)
print(_text)

bot = UploadRobot(image_path,
                        description = _text,
                        useFilename = image_name,
                        keepFilename = True,
                        verifyDescription = False,
                        ignoreWarning = True,
                        targetSite = commons_site)
bot.run()

os.remove(image_path) 

2018-01-10 13:09:50,908 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.7 (KHTML, like Gecko) Chrome/7.0.514.0 Safari/534.'}
2018-01-10 13:09:50,910 root   DEBUG    Using proxy: 185.71.80.3:8888 | FreeProxy


Image to download: http://www.lamoncloa.gob.es/multimedia/fototeca/Paginas/ImageF.axd?img=70d716c531
Image downloaded to: /home/paws/Fernández de la Vega preside la reunión sobre la violencia de genero con los subdelegados de Gobierno.jpg
Trying to download
RR Status 200


2018-01-10 13:10:07,558 pywiki WARNING  url as string is deprecated. Use an iterable instead.
2018-01-10 13:10:07,570 pywiki VERBOSE  Found 1 commons:commons processes running, including this one.
2018-01-10 13:10:07,572 requests_oauthlib.oauth1_auth DEBUG    Signing request <PreparedRequest [POST]> using client <Client client_key=0a73e346a40b07262b6e36bdba01cba4, client_secret=****, resource_owner_key=5ca4fafa5dd85bb0ecf516d96ad6bf07, resource_owner_secret=****, signature_method=HMAC-SHA1, signature_type=AUTH_HEADER, callback_uri=None, rsa_key=None, verifier=None, realm=None, encoding=utf-8, decoding=utf-8, nonce=None, timestamp=None>
2018-01-10 13:10:07,576 requests_oauthlib.oauth1_auth DEBUG    Including body in call to sign: True
2018-01-10 13:10:07,577 oauthlib.oauth1.rfc5849 DEBUG    Collected params: [('oauth_nonce', '77764412245103813451515589807'), ('oauth_timestamp', '1515589807'), ('oauth_version', '1.0'), ('oauth_signature_method', 'HMAC-SHA1'), ('oauth_consumer_key', '0a73

2018-01-10 13:10:07,719 requests_oauthlib.oauth1_auth DEBUG    Updated body: b'iiprop=timestamp%7Cuser%7Ccomment%7Curl%7Csize%7Csha1%7Cmime%7Cmetadata%7Carchivename&titles=File%3AFern%C3%A1ndez+de+la+Vega+preside+la+reuni%C3%B3n+sobre+la+violencia+de+genero+con+los+subdelegados+de+Gobierno.jpg&prop=imageinfo&action=query&indexpageids=&continue=&iilimit=500&meta=userinfo&uiprop=blockinfo%7Chasmsg&maxlag=5&format=json'


Image downloaded and accessed
=={{int:filedesc}}==
{{Information
|description={{es|1=María Teresa Fernández de la Vega, vicepresidenta primera, ministra de la Presidencia y portavoz del Gobierno, preside la reunión sobre la violencia de genero con los subdelegados de Gobierno junto a José Enrique Serrano Martínez, director del Gabinete del Presidente.}}
|date=2007-04-10
|source=[http://www.lamoncloa.gob.es/multimedia/fototeca/Paginas/Imagen.aspx?img=08039dc131 Fototeca: Fernández de la Vega preside la reunión sobre la violencia de genero con los subdelegados de Gobierno]
|author=Ministerio de la Presidencia. Gobierno de España (Pool Moncloa)
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
[[Category:María Teresa Fernández de la Vega]]
[[Category:José Enrique Serrano Martínez]]
[[Category:April 2007 in Madrid]]
[[Category:Palacio de la Moncloa]]


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=María Teresa Fernández de la Vega, vicepresidenta primera, ministra de la Presidencia y portavoz del Gobierno, preside la reunión sobre la violencia de genero con los subdelegados de Gobierno junto a José Enrique Serrano Martínez, director del Gabinete del Presidente.}}
|date=2007-04-10
|source=[http://www.lamoncloa.gob.es/multimedia/fototeca/Paginas/Imagen.aspx?img=08039dc131 Fototeca: Fernández de la Vega preside la reunión sobre la violencia de genero con los subdelegados de Gobierno]
|author=Ministerio de la Presidencia. Gobierno de España (Pool Moncloa)
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
[[Category:María Teresa Fernández de la Vega]]
[[Category:José Enrique Serrano Martínez]]
[[Category:April 2007 in Madrid]]
[[Category:Palacio de la Moncloa]]
2018-01-10 13:10:07,792 pywiki INFO    

2018-01-10 13:10:08,204 requests_oauthlib.oauth1_auth DEBUG    Updated body: b'action=query&meta=tokens%7Cuserinfo&type=csrf&uiprop=blockinfo%7Chasmsg&rawcontinue=&maxlag=5&format=json'
Sleeping for 9.5 seconds, 2018-01-10 13:10:08
2018-01-10 13:10:08,256 pywiki INFO     Sleeping for 9.5 seconds, 2018-01-10 13:10:08
2018-01-10 13:10:17,880 requests_oauthlib.oauth1_auth DEBUG    Signing request <PreparedRequest [POST]> using client <Client client_key=0a73e346a40b07262b6e36bdba01cba4, client_secret=****, resource_owner_key=5ca4fafa5dd85bb0ecf516d96ad6bf07, resource_owner_secret=****, signature_method=HMAC-SHA1, signature_type=AUTH_HEADER, callback_uri=None, rsa_key=None, verifier=None, realm=None, encoding=utf-8, decoding=utf-8, nonce=None, timestamp=None>
2018-01-10 13:10:17,881 requests_oauthlib.oauth1_auth DEBUG    Including body in call to sign: False
2018-01-10 13:10:17,885 oauthlib.oauth1.rfc5849 DEBUG    Collected params: [('oauth_nonce', '156207289968339573231515589817'), ('oauth